In [1]:
import matplotlib.pyplot as plot
import matplotlib.animation
import numpy as np

\begin{align}
    \left|\Psi(x,t)\right|^2 &= \sum_{n,m=1}^\infty e^{i\left(E_n - E_m\right)t}c_n c_m \psi_n(x)\psi_m(x)\\
                             &= \sum_{n=1}^\infty c_n^2 \psi_n^2(x) + 
                                \sum_{n < m}^\infty 2\cos\left[\left(E_n - E_m\right)t\right] c_n c_m \psi_n(x)\psi_m(x)\\
                             &= \sum_{n,m=1}^\infty\cos\left[\left(E_n - E_m\right)t\right] c_n c_m \psi_n(x)\psi_m(x)
\end{align}

where the last line is obtained by relying on $ e^{i\theta}+e^{-i\theta} = 2 \cos\left(\theta\right)$.
                          

In [2]:
L = 1
mass = 1
c = np.array([1/np.sqrt(2), 1/np.sqrt(2)])

def egy(p):
    return p * p * np.pi * np.pi / (2 * mass * L)

def time_dependent_coefficient(c, t):
    rv = np.zeros((len(c), ), dtype=complex)
    for i in range(len(c)):
        rv[i] = c[i] * np.exp(-1.0j * egy(i+1) * t)
    return rv

energies = np.array([egy(i + 1) for i in range(len(c))])
ct = time_dependent_coefficient(c, 0.0)
print(energies)
print(ct)

[ 4.9348022 19.7392088]
[0.70710678+0.j 0.70710678+0.j]


In [29]:
def time_dependent_density(p0, dt):
    rv = np.zeros(p0.shape, dtype=complex)
    for i in range(len(c)):
        ei = egy(i + 1)
        for j in range(len(c)):
            ej = egy(j + 1)
            rv[i, j] = p0[i, j] * np.exp(-1.0j * (ei - ej) * dt)
    return rv

pt = [np.outer(ct, np.conj(ct))]
for t in range(1, 10):
    pt.append(time_dependent_density(pt[t - 1], 0.1))

print(pt[2])

[[ 0.5       +0.j         -0.49185805+0.08986468j]
 [-0.49185805-0.08986468j  0.5       +0.j        ]]


In [4]:
def spatial_wavefunction(p, x):
    return np.sqrt(2.0 / L) * np.sin(p * np.pi * x/ L)

grid = np.linspace(0, L, 100)
real_space_wf = [np.array([spatial_wavefunction(i + 1, x) for x in grid]) for i in range(len(c))]

In [5]:
from ipywidgets import interact, widgets

def probability_density(p, real_space_wf):
    value_at_x = np.zeros(real_space_wf[0].shape, dtype=complex)
    n_rows, n_cols = p.shape
    for i in range(n_rows):
        psi_i = real_space_wf[i]
        for j in range(n_cols):
            psi_ij = psi_i * real_space_wf[j]
            value_at_x += p[i,j] * psi_ij
    
    return value_at_x

def plot_probability(t):
    fig, ax = plot.subplots()
    real_space_pt = probability_density(pt[t], real_space_wf)
    ax.plot(grid, real_space_pt)    
    ax.set_ylim([0, 4])
    plot.show()
    
interact(plot_probability, t=widgets.IntSlider(min=0, max=(len(pt) - 1), step=1, value=0))

10


interactive(children=(IntSlider(value=0, description='t', max=9), Output()), _dom_classes=('widget-interact',)…

<function __main__.plot_probability(t)>

In [22]:
def plot_probability2(t):
    fig, ax = plot.subplots()
    real_space_pt = probability_density(pt2[t], real_space_wf)
    ax.plot(grid, real_space_pt)    
    ax.set_ylim([0, 4])
    plot.show()

interact(plot_probability2, t=widgets.IntSlider(min=0, max=(len(pt2) - 1), step=1, value=0))

interactive(children=(IntSlider(value=0, description='t', max=9), Output()), _dom_classes=('widget-interact',)…

<function __main__.plot_probability2(t)>